# Import libraries and check GPU

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes

!pip install xformers

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-qoh6_4ow/unsloth_7a53fb0792924850971e78acf534350a
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-qoh6_4ow/unsloth_7a53fb0792924850971e78acf534350a
  Resolved https://github.com/unslothai/unsloth.git to commit 64bb8cfd512a9dcd860d21563b624676f7432ec5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
# Para limpiar cache GPU:
torch.cuda.empty_cache()

import gc
gc.collect()

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


# Step 1: load model with unsloth library

Solo se pasa los LoRA adapters, ya que unsloth se encarga de unirlos con el modelo base con el que se entreno (esto viene en un json en el repositorio de Hugging Face en cuestión)

In [5]:
from unsloth import FastLanguageModel
from datasets import load_dataset

# Import model pretained with unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "bfrenan/Llama3-log-to-ttp-lora-adapters_2", # Aqui se pone directamente el LoRA adapter
    max_seq_length = 2048,
    dtype = None, #autodetection
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Import dataset
dataset = load_dataset("bfrenan/log_to_TTP_split", split="test")
print(dataset)

adapter_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Generating train split:   0%|          | 0/22301 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2478 [00:00<?, ? examples/s]

Dataset({
    features: ['source', 'full_log', 'description', 'level', 'technique', 'tactic'],
    num_rows: 2478
})


# Step 2: Define inference function

In [109]:
def inference(row):
    prompt_format='''<|begin_of_text|><|start_header_id|>system<|end_header_id|>{}<|eot_id|><|start_header_id|>user<|end_header_id|>{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>{}'''

    inputs = tokenizer(
    [
        prompt_format.format(
            '''You are a cybersecurity assistant. Your mission is, given a system log, return the possible TTP code from MITRE ATT&CK.''', # instruction
            f'''Source: {dataset[row]['source']}; Description: {dataset[row]['description']}; Level: {dataset[row]['level']}; Full log: {dataset[row]['full_log']}''', # input
            '''''', # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
    return tokenizer.batch_decode(outputs)[0] # se pone [0] porque tokenizer devuelva lista de un elemento, y se quiere string

print(inference(54))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a cybersecurity assistant. Your mission is, given a system log, return the possible TTP code from MITRE ATT&CK.<|eot_id|><|start_header_id|>user<|end_header_id|>Source: /var/log/auth.log; Description: Successful sudo to ROOT executed.; Level: 3; Full log: Oct 18 23:43:57 sohaib-virtual-machine sudo:   sohaib : TTY=pts/11 ; PWD=/home/sohaib ; USER=root ; COMMAND=/usr/bin/sh -c 'printf "command_start_%s" "m6pUkaK9"; file /snap/core22/current/usr/share/bash-completion/completions/sqlite3; printf "command_done_%s" "_z0Tkj5e"'<|eot_id|><|start_header_id|>assistant<|end_header_id|>The tactics are ["TA0004","TA0005"] and the techniques are ["T1548.003"]<|eot_id|>


# Step 3: Loop and check if model predicts well in test data

It will be evaluated with accuracy metric = correct predictions / total predictions.

The principal problem is that in one prediction there are more than one TTP, so how can I determine if the answer is correct or no?

There is a way: taking the total numbers of tecnics and tactics instead of the total number of rows. In this way, there will be more "predictions" than rows (inputs prompts), because in each prompt the model can return more than one TTP.

There will be:
- Tactic accuracy
- Technique accuracy
- Total accuracy: considering weights 50% for tactic and 50% fot tecnics. So it will be equal to:
0.5 · tactic_accuracy + 0.5 · techcnique_accuracy



#### **PROBLEMA:**

Los TTP están en formato string (tanto en el dataset como en la respuesta del model). Primero se ha de hacer una lista con los TTP de cada row y verificarlo con cada respuesta mediante expresiones regulares.

In [69]:
import re

# Se consigue una lista de TTPs bien separadas a partir de un string
def limpiar(ttp):
    ttp=ttp.replace('"','').strip('[]') # Para quitar "" y []
    lista_ttp=ttp.split(',') # Para obtener una lista con las TTP
    return lista_ttp

# Para coger ambas TTP de la respuesta. Se compila regex por rapidez
regex=re.compile('\[\"T.+?\"\]')
# Se definen las variables donde se guardaran las ttps totales y acertadas para luego hacer el accuracy
totales_tactics=0
totales_techniques=0
acertadas_tactics=0
acertadas_techniques=0
# Se recorre el dataset a la vez que se hace inferencia
for i in range(len(dataset)):
    # Coger TTPs de la fila i del dataset
    dataset_tactics=limpiar(dataset[i]["tactic"])
    dataset_techniques=limpiar(dataset[i]["technique"])
    # Coger TTPs de la respuesta i del LLM
    tactic, technique=regex.findall(inference(i)) # Paso adicional: matchear regex de la respuesta del LLM
    llm_tactics=limpiar(tactic)
    llm_techniques=limpiar(technique)
    # Comprobar cuantos TTP ha acertado el modelo. Vamos calculando de manera acumulativa las tecnicas/tacticas totales y acertadas
    acertadas_tactics+=len([ttp for ttp in llm_tactics if ttp in dataset_tactics]) # comprension de lista para ver las diferencias
    acertadas_techniques+=len([ttp for ttp in llm_techniques if ttp in dataset_techniques])
    totales_tactics+=len(dataset_tactics)
    totales_techniques+=len(dataset_techniques)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [104]:
print("TACTICAS -> Acertadas: {}, Totales:{}".format(acertadas_tactics, totales_tactics))
print("TECNICAS -> Acertadas: {}, Totales:{}".format(acertadas_techniques, totales_techniques))
accuracy_tactics=acertadas_tactics/totales_tactics
print("Accuracy tactics: ", accuracy_tactics)
accuracy_techniques=acertadas_techniques/totales_techniques
print("Accuracy techniques: ", accuracy_techniques)
accuracy=0.5*accuracy_tactics+0.5*accuracy_techniques
print("Accuracy total: ", accuracy)
# Recordemos que el dataset test tiene 2478 ejemplos

TACTICAS -> Acertadas: 7270, Totales:7270
TECNICAS -> Acertadas: 5274, Totales:5274
Accuracy tactics:  1.0
Accuracy techniques:  1.0
Accuracy total:  1.0
